In [24]:
from __future__ import unicode_literals
from imposm.parser import OSMParser
import json

import sys

import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt

SRC_FILENAME = 'central-fed-district-latest.osm.pbf'
SRC_FILENAME = 'crimean-fed-district-latest.osm.pbf'

df = pd.DataFrame()
df['test'] = ['one', 'one','two','three','four','three']
df.append({'test':'ddd'}, ignore_index=True)
df.head()

,test
0,one
1,one
2,two
3,three
4,four


In [2]:
# simple class that handles the parsed OSM data.
class StadiumCounter(object):
    stadium = 0
    
    def nodes(self, nodes):

        for id, tags, (lon, lat) in nodes:
            if 'leisure' in tags:
                if 'stadium' in tags['leisure']:
                    if len(tags)>4:
                      self.stadium += 1
                      print('----------------node--------------------------')
                      print (json.dumps(tags, indent=4, sort_keys=True, ensure_ascii=False))

    def ways(self, ways):
    
        for osmid, tags, refs in ways:
            if 'leisure' in tags:
                if 'stadium' in tags['leisure']:
                    if len(tags)>4:
                      self.stadium += 1
                      print('----------------ways------------------------')
                      print (json.dumps(osmid, indent=4, sort_keys=True, ensure_ascii=False))
                      print (json.dumps(tags, indent=4, sort_keys=True, ensure_ascii=False))
                        
    def coords(self, coords):
        for (osm_id, lon, lat) in coords:
            if osm_id in nodes_ids:
                lon = int(lon + 1000) - 1000
                lat = int(lat + 1000) - 1000
                nodes_counts[(lon, lat)] += 1


# instantiate counter and parser and start parsing
counter = StadiumCounter()
p = OSMParser(concurrency=4, 
              ways_callback=counter.ways,  
#                nodes_callback=counter.nodes,
              coords_callback=counter.coords
             )
p.parse(SRC_FILENAME)

# p = OSMParser(concurrency=4, coords_callback=count_coords)
# p.parse(SRC_FILENAME)

# done
print counter.stadium

NameError: global name 'nodes_ids' is not defined

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

'''
Simple script to create geojson from osm files.
Prints geojson features line by line.
Currently we don't support relations.
Requires (unmodified) imposm and imposm.parser
Inspired by https://github.com/emka/OSMCouch
'''

from multiprocessing import JoinableQueue
from imposm.parser import OSMParser
from imposm.reader import CacheWriterProcess
from imposm.cache import OSMCache

import sys
import io
import json


def generateCache(filename = SRC_FILENAME):
    cache = OSMCache('.')
    coords_queue = JoinableQueue(512)
    coords_writer = CacheWriterProcess(coords_queue, cache.coords_cache, None, marshaled_data=True)
    coords_writer.start()

    parser = OSMParser(coords_callback=coords_queue.put)

    parser.parse(filename)

    coords_queue.put(None)
    coords_writer.join()
    return cache

class Handler(object):

    def __init__(self, cache, output):
        self.coordsCache = cache.coords_cache(mode='r')
        self.outputFile = io.open(output, mode='w')

    def nodeToGEOJson(self, node):
        id, tags, (lon, lat) = node
        return unicode(json.dumps({ 'id': id, 'type': 'Feature', 'geometry': { 'type': 'Point', 'coordinates': [lon, lat] } , 'properties': tags }, ensure_ascii=False) + '\n')

    def wayToGEOJson(self, way):
        id, tags, nodes = way
        coords = self.coordsCache.get_coords(nodes)
        return unicode(json.dumps({ 'id': id, 'type': 'Feature', 'geometry': { 'type': 'LineString', 'coordinates': coords }, 'properties': tags }, ensure_ascii=False) + '\n')

    def nodes_callback(self, nodes):
        for node in nodes:
            self.outputFile.write(self.nodeToGEOJson(node))
            #self.file.write(self.nodeToGEOJson(node))

    def ways_callback(self, ways):
        for osmid, tags, refs, way in ways:
            if 'leisure' in tags:
                if 'stadium' in tags['leisure']:
                    if len(tags)>4:
                      self.stadium += 1
                      print('---------------------------------------------------')
                      print (json.dumps(osmid, indent=4, sort_keys=True, ensure_ascii=False))
                      print (json.dumps(tags, indent=4, sort_keys=True, ensure_ascii=False))
                      self.outputFile.write(self.wayToGEOJson(way))


def main(filename, output):
    #cache = generateCache(filename)
    cache = OSMCache('.')
    coords_queue = JoinableQueue(512)
    coords_writer = CacheWriterProcess(coords_queue, cache.coords_cache, None, marshaled_data=True)
    coords_writer.start()

    handler = Handler(cache, output)
    parser = OSMParser(coords_callback=coords_queue.put,
#             nodes_callback=handler.nodes_callback,
            ways_callback=handler.ways_callback
            )
    parser.parse(filename)

    coords_queue.put(None)
    coords_writer.join()



main(SRC_FILENAME, 'output.json')